# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://preview.documentation.bedrock.aws.dev/Documentation/BedrockUserGuide.pdf

## 1. role setting (adding trust relationship)

### 1.1. role check

In [2]:
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20230712T200237


### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 2개의 권한 추가

![nn](../imgs/policy.jpg)

### 1.3. policy
- 아래와 같이 인라인 정책 생성 (Add permissions - Create inline policy)
- policy name: bedrock
![nn](../imgs/inline-policy.jpg)

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Sid": "BedrockFullAccess"
        }
    ]
}

### 1.4. Trust relationship
![nn](../imgs/trust-relationship.jpg)

In [ ]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

## 2. Install python SDK for bedrock

In [4]:
!wget https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/Python/bedrock-runtime.zip ## prev: bedrock-python-sdk.zip
!unzip bedrock-runtime.zip -d bedrock-sdk
!rm -rf bedrock-runtime.zip

--2023-09-27 01:22:30--  https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/Python/bedrock-runtime.zip
Resolving d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)... 108.138.82.216, 108.138.82.42, 108.138.82.211, ...
Connecting to d2eo22ngex1n9g.cloudfront.net (d2eo22ngex1n9g.cloudfront.net)|108.138.82.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252012954 (240M) [application/zip]
Saving to: ‘bedrock-runtime.zip’

bedrock-runtime.zip 100%[===================>] 240.34M  53.5MB/s    in 4.5s    

2023-09-27 01:22:37 (53.5 MB/s) - ‘bedrock-runtime.zip’ saved [252012954/252012954]

Archive:  bedrock-runtime.zip
   creating: bedrock-sdk/reviews/
  inflating: bedrock-sdk/.functional-crt  
  inflating: bedrock-sdk/AWSCLI32.msi  
  inflating: bedrock-sdk/models-starfort-report.json  
  inflating: bedrock-sdk/awscli-bundle.zip  
  inflating: bedrock-sdk/boto3-1.28.55.tar.gz  
  inflating: bedrock-sdk/AWSCLI32PY3.msi  
  inflating: bedrock-sdk/s3t

In [5]:
install_needed = True

In [6]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U ./bedrock-sdk/botocore-*-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/boto3-*-py3-none-any.whl
    !{sys.executable} -m pip install -U ./bedrock-sdk/awscli-*-py3-none-any.whl
    !{sys.executable} -m pip install -U langchain #==0.0.286
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    #!{sys.executable} -m pip install -U urllib
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !rm -rf bedrock-sdk

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
  Using cached sagemaker-2.188.0-py2.py3-none-any.whl
  Obtaining dependency information for urllib3<1.27,>=1.25.4 from https://files.pythonhosted.org/packages/c5/05/c214b32d21c0b465506f95c4f28ccbcba15022e000b043b72b3df7728471/urllib3-1.26.16-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.16-py2.py3-none-any.whl.metadata (48 kB)
Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.184.0
    Uninstalling sagemaker-2.184.0:
      Successfully uninstalled sagemaker-2.184.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you h

## 3. Check setting
아래 메시지가 뜨면 정상 <BR>
![nn](../imgs/check.png)


In [1]:
import boto3
import langchain

In [2]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")

langchain version check: 0.0.302
boto3 version check: 1.28.55


In [3]:
is_internal_use = True ## if 고객: False, aws직원: True

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [4]:
bedrock_region = "us-east-1" ## "us-east-1" or "us-west-2"

In [5]:
if bedrock_region == "us-east-1":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://bedrock-runtime.us-east-1.amazonaws.com" if is_internal_use else None
    }
elif bedrock_region == "us-west-2":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://prod.us-west-2.frontend.bedrock.aws.dev" if is_internal_use else None
    }

In [6]:
bedrock_config

{'region_name': 'us-east-1',
 'endpoint_url': 'https://bedrock-runtime.us-east-1.amazonaws.com'}

In [7]:
if is_internal_use:
    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name=bedrock_config["region_name"],
        endpoint_url=bedrock_config["endpoint_url"]
    )
else:
    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name=bedrock_config["region_name"]
    ) 

In [8]:
# output_text = bedrock_client.list_foundation_models()
# output_text 

## 4. generation

In [9]:
from termcolor import colored
from langchain.llms.bedrock import Bedrock

### Titan

In [10]:
llm = Bedrock(
    model_id="amazon.titan-tg1-large",
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    }
)

In [11]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [12]:
print (colored(llm(prompt), "green"))


SageMaker is a fully managed service from Amazon that provides every developer and data scientist the best practices to build, train, and deploy high-quality machine learning (ML) models quickly. With SageMaker, there is no need to select and provision servers, configure ML infrastructure, or manage storage and security. Just use APIs to create, train, and deploy models fully managed in the cloud.


### Claude v2

In [13]:
llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=bedrock_client,
    model_kwargs={
        "max_tokens_to_sample":512,
        "stop_sequences":[],
        "temperature":0,
        "top_p":0.9
    }
)

In [14]:
prompt = "\n\nHuman: Please let us know SageMaker's advantages in 100 words \n\nAssistant:"
#prompt = "SageMaker의 장점을 100단어 미만으로 설명해줘"

In [15]:
print (colored(llm(prompt), "green"))

 Here is a 100 word summary of some of SageMaker's key advantages:

SageMaker makes it easy to build, train, and deploy machine learning models at scale. It provides pre-built algorithms and frameworks as well as fully managed infrastructure to quickly get models into production. Automated hyperparameter tuning saves time by finding the best model configurations. SageMaker Studio provides a visual IDE for the full ML workflow. And SageMaker Neo compiles models for faster inference. The service is fully managed so you don't have to worry about infrastructure management. These capabilities help data scientists be more productive and get models to market faster.
